In [47]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import gensim
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [40]:
!pip install --user gensim

In [38]:
df=pd.read_table('outcomes.tsv')
df.head(3)

,auction_id,product_id,item,desc,retail,price,finalprice,bidincrement,bidfee,winner,placedbids,freebids,endtime_str,flg_click_only,flg_beginnerauction,flg_fixedprice,flg_endprice
0,86827,10009602,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,13.35,13.35,15,75,Racer11,26,0,19:52 PDT 09-16-2008,0,0,0,0
1,87964,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,74.70,74.70,15,75,Cemo23,65,0,11:17 PDT 08-28-2008,0,0,0,0
2,87965,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,83.10,83.10,15,75,Jacobsonnich,94,0,22:52 PDT 11-07-2008,0,1,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121419 entries, 0 to 121418
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   auction_id           121419 non-null  int64  
 1   product_id           121419 non-null  int64  
 2   item                 121419 non-null  object 
 3   desc                 121419 non-null  object 
 4   retail               121419 non-null  float64
 5   price                121419 non-null  float64
 6   finalprice           121419 non-null  float64
 7   bidincrement         121419 non-null  int64  
 8   bidfee               121419 non-null  int64  
 9   winner               121419 non-null  object 
 10  placedbids           121419 non-null  int64  
 11  freebids             121419 non-null  int64  
 12  endtime_str          121419 non-null  object 
 13  flg_click_only       121419 non-null  int64  
 14  flg_beginnerauction  121419 non-null  int64  
 15  flg_fixedprice   

In [16]:
df['product_id'].value_counts()

10012220    6658
10006485    5893
10011707    3283
10011706    2365
10009318    2297
            ... 
10013942       1
10011465       1
10009745       1
10013288       1
10009933       1
Name: product_id, Length: 2081, dtype: int64

In [17]:
df['auction_id'].value_counts()

86827     1
196684    1
196696    1
196695    1
196694    1
         ..
146538    1
146537    1
146536    1
146535    1
259070    1
Name: auction_id, Length: 121419, dtype: int64

In [28]:
df['winner'].value_counts()

HH0812          131
Oglalasioux     117
Toertchen       116
Koboldwicht     112
Schlumpf01      111
               ... 
Terrigirl         1
Jsmnl             1
Sadettin5454      1
R083r77           1
Fublin            1
Name: winner, Length: 45787, dtype: int64

In [44]:
def preprocess_text(text: str, remove_stopwords:bool) -> str:
    """Function that cleans the input text by going to:
    - remove links
    - remove special characters
    - remove numbers
    - remove stopwords
    - convert to lowercase
    - remove excessive white spaces
    Arguments:
        text (str): text to clean
        remove_stopwords (bool): whether to remove stopwords
    Returns:
        str: cleaned text
    """
    text=re.sub(r"http\S+","",text)
    text=re.sub("[^A-Za-z]+"," ",text)
    if remove_stopwords:
      tokens=nltk.word_tokenize(text)
      tokens=[w.lower().strip() for w in tokens if not w.lower() in stopwords.words("italian")]
      return tokens

In [46]:
df1=df[['item','desc']]
df2=df1.apply(lambda x:','.join(x.astype(str)),axis=1)
df_clean=pd.DataFrame({"clean":df2})
product=[row.split(',') for row in df_clean['clean']]
product[0:2]

[['sony-ericsson-s500i-unlocked-mysterious-',
  'Sony Ericsson S500i Unlocked Mysterious Green'],
 ['psp-slim-lite-sony-piano-black', 'PSP Slim & Lite Sony Piano Black']]

In [48]:
model = Word2Vec(product, min_count=1, vector_size= 50, workers=3, window=3, sg=1)

In [49]:
model.wv['sony-ericsson-s500i-unlocked-mysterious-']

array([ 0.49225262, -0.1925792 ,  0.19929004,  0.4875002 , -0.1331615 ,
       -0.3566722 , -0.37588403,  0.03162694,  0.02376109, -0.339252  ,
       -0.26604903,  0.067588  ,  0.00670887,  0.4130529 , -0.4313137 ,
        0.25339264, -0.3496237 , -0.12169349, -0.07719272, -0.4543877 ,
        0.26635528, -0.3354242 ,  0.31672952, -0.31204262, -0.06072189,
       -0.4239419 , -0.18003818, -0.13569592, -0.1713079 ,  0.40775466,
       -0.5097931 , -0.23997355,  0.09303638, -0.4049906 , -0.07348803,
       -0.46615097, -0.17026584,  0.0748322 ,  0.05800678, -0.161557  ,
       -0.12705661, -0.5157769 ,  0.33144906,  0.07830773,  0.0193123 ,
       -0.09019019, -0.04874843,  0.03913241, -0.11609981, -0.05589892],
      dtype=float32)

In [50]:
model.wv.most_similar('sony-ericsson-s500i-unlocked-mysterious-')

[('Whirlpool Duet 4.4 Cu. Ft. 12-Cycle Washer', 0.7370995879173279),
 ('Coby 2GB Flash MP3 Player with FM (Red)', 0.7293650507926941),
 ('samsung-pleomax-psp-7000b-usb-speakers', 0.7180401086807251),
 ('Summer Sports Paradise (Wii)', 0.7149981260299683),
 ('Professor Layton and the Diabolical Box (DS)', 0.714982271194458),
 ('dyson-dc17-animal-cyclone-upright-vacuum', 0.7146457433700562),
 ('Pro-Evolution Soccer 2010 (PS3)', 0.7118538022041321),
 ('Escada Sunset Heat EDT Spray 3.4 Oz', 0.7116205096244812),
 ('Fable II (Xbox 360)', 0.7110999226570129),
 ('Sony Ericsson U1i Satio Idou Unlocked Phone', 0.7108854055404663)]

In [51]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [52]:
def cosine_distance (model, word,target_list , num) :
    cosine_dict ={}
    word_list = []
    a = model.wv[word]
    for item in target_list :
        if item != word :
            b = model.wv[item]
            cos_sim = dot(a, b)/(norm(a)*norm(b))
            cosine_dict[item] = cos_sim
    dist_sort=sorted(cosine_dict.items(), key=lambda dist: dist[1],reverse = True)
    for item in dist_sort:
        word_list.append((item[0], item[1]))
    return word_list[0:num]

In [53]:
item = list(df.item.unique())
cosine_distance (model,'sony-ericsson-s500i-unlocked-mysterious-',item,5)

[('samsung-pleomax-psp-7000b-usb-speakers', 0.7180401),
 ('dyson-dc17-animal-cyclone-upright-vacuum', 0.7146457),
 ('sony-dcr-sr47-hard-disk-drive-handycam-c', 0.7108286),
 ('esprit-full-activity-black-mens-watch', 0.71050894),
 ('call-of-duty-modern-warfare-2-xbox-360-', 0.7079246)]

In [56]:
df['product']= df['item']+ " " + df['desc']
df.head(3)

,auction_id,product_id,item,desc,retail,price,finalprice,bidincrement,bidfee,winner,placedbids,freebids,endtime_str,flg_click_only,flg_beginnerauction,flg_fixedprice,flg_endprice,product
0,86827,10009602,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,13.35,13.35,15,75,Racer11,26,0,19:52 PDT 09-16-2008,0,0,0,0,sony-ericsson-s500i-unlocked-mysterious- Sony ...
1,87964,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,74.70,74.70,15,75,Cemo23,65,0,11:17 PDT 08-28-2008,0,0,0,0,psp-slim-lite-sony-piano-black PSP Slim & Lite...
2,87965,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,83.10,83.10,15,75,Jacobsonnich,94,0,22:52 PDT 11-07-2008,0,1,0,0,psp-slim-lite-sony-piano-black PSP Slim & Lite...


In [57]:
df3=df.drop(columns=['auction_id','product_id','item','desc','retail','price','finalprice',
                     'bidincrement','bidfee','winner','placedbids','freebids','endtime_str',
                     'flg_click_only','flg_beginnerauction','flg_fixedprice','flg_endprice'])
df3=df3.reset_index(drop=True)
df3

,product
0,sony-ericsson-s500i-unlocked-mysterious- Sony ...
1,psp-slim-lite-sony-piano-black PSP Slim & Lite...
2,psp-slim-lite-sony-piano-black PSP Slim & Lite...
3,sony-ericsson-s500i-unlocked-mysterious- Sony ...
4,sony-ericsson-s500i-unlocked-mysterious- Sony ...
...,...
121414,300-bids-voucher 300 Bids Voucher
121415,300-bids-voucher 300 Bids Voucher
121416,300-bids-voucher 300 Bids Voucher
121417,300-bids-voucher 300 Bids Voucher


In [62]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.4 MB/s eta 0:00:00


In [63]:
!pip install BERTopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 45.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039184 sha256=14c2c187297508ee4cbc3918418701f2d4c369c3f7e0b4bf4fccd6f46d92c7f4
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca8

In [64]:
from bertopic import BERTopic

In [65]:
products=df['product'].to_list()
products[:3]

['sony-ericsson-s500i-unlocked-mysterious- Sony Ericsson S500i Unlocked Mysterious Green',
 'psp-slim-lite-sony-piano-black PSP Slim & Lite Sony Piano Black',
 'psp-slim-lite-sony-piano-black PSP Slim & Lite Sony Piano Black']

In [66]:
model=BERTopic(verbose=True,embedding_model='paraphrase-MiniLM-L3-v2')
topic,probs=model.fit_transform(products)

Batches:   0%|          | 0/3795 [00:00<?, ?it/s]

2023-08-09 20:40:09,139 - BERTopic - Transformed documents to Embeddings
2023-08-09 20:46:11,759 - BERTopic - Reduced dimensionality
2023-08-09 20:46:27,458 - BERTopic - Clustered reduced embeddings


In [67]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2677,-1_ts8gjfv10_wave_8gb_80,"[ts8gjfv10, wave, 8gb, 80, cobalt, cash, dr, m...",[transcend-ts8gjfv10-usb-2-0-8gb-flash-dr Tran...
1,0,6658,0_20_bids_voucher_,"[20, bids, voucher, , , , , , , ]","[20-bids-voucher 20 Bids Voucher, 20-bids-vouc..."
2,1,5935,1_sports_console_wii_nintendo,"[sports, console, wii, nintendo, , , , , , ]",[wii-nintendo-console-wii-sports Wii | Nintend...
3,2,3283,2_50_bids_voucher_,"[50, bids, voucher, , , , , , , ]","[50-bids-voucher 50 Bids Voucher, 50-bids-vouc..."
4,3,2365,3_300_bids_voucher_,"[300, bids, voucher, , , , , , , ]","[300-bids-voucher 300 Bids Voucher, 300-bids-v..."
...,...,...,...,...,...
1272,1271,10,1271_remote_play_with_wii,"[remote, play, with, wii, , , , , , ]",[wii-play-with-wii-remote Wii Play with Wii Re...
1273,1272,10,1272_cash_80__,"[cash, 80, , , , , , , , ]","[-80-cash- $80 Cash!, -80-cash- $80 Cash!, -80..."
1274,1273,10,1273_luxio_talent_32gb_super,"[luxio, talent, 32gb, super, blac, drive, flas...",[super-talent-luxio-32gb-flash-drive-blac Supe...
1275,1274,10,1274_tt_talea_st_saeco,"[tt, talea, st, saeco, machi, espresso, machin...",[saeco-s-tt-st-talea-touch-espresso-machi Saec...


In [68]:
model.visualize_barchart(top_n_topics=5)

In [71]:
similar,similarity=model.find_topics("s500i")
print(similar)
most_similar=similar[0]
print("Most similar topic:\n{}".format(model.get_topic(most_similar)))
print('similarity Score:{}'.format(similarity[0]))

[1005, 744, 672, 980, 459]
Most similar topic:
[('150', 0.15011545799142725), ('t1i', 0.08589414457746684), ('eos', 0.08467485294897577), ('rebel', 0.038744943627898834), ('55mm', 0.035240033177209466), ('canon', 0.030044950795535614), ('freebids', 0.014805597395426804), ('18', 0.0136144077319864), ('', 1e-05), ('', 1e-05)]
similarity Score:0.3371555805206299


In [72]:
similar,similarity=model.find_topics("dyson")
print(similar)
most_similar=similar[0]
print("Most similar topic:\n{}".format(model.get_topic(most_similar)))
print('similarity Score:{}'.format(similarity[0]))

[584, 605, 711, 474, 701]
Most similar topic:
[('floors', 0.3491606801021813), ('dc25', 0.3491606801021813), ('upright', 0.273789465730392), ('dyson', 0.26038061618774994), ('vacuu', 0.21831485585915422), ('ball', 0.18626311115943264), ('cleaner', 0.12127722956107796), ('vacuum', 0.09236925258252618), ('all', 0.08003183435685812), ('', 1e-05)]
similarity Score:0.6174430847167969


In [73]:
similar,similarity=model.find_topics("samsung-pleomax-psp-7000b-usb-speakers")
print(similar)
most_similar=similar[0]
print("Most similar topic:\n{}".format(model.get_topic(most_similar)))
print('similarity Score:{}'.format(similarity[0]))

[514, 248, 1222, 109, -1]
Most similar topic:
[('pleomax', 0.4250314903130693), ('7000b', 0.4250314903130693), ('speakers', 0.15875728614488394), ('psp', 0.1039321990517176), ('samsung', 0.025596131953280924), ('usb', 0.011406749296874377), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05)]
similarity Score:0.9714629650115967
